In [2]:
import tensorflow as tf
from keras.layers import Conv1D,Conv1DTranspose,Concatenate,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError
import numpy as np
import IPython.display
import glob
from tqdm import tqdm
import librosa.display
import matplotlib.pyplot as plt

In [ ]:
import os
print(os)


<built-in function listdrives>


In [ ]:
noisy_train = np.load('')
clean_train = np.load('')

In [9]:

def get_dataset(x_train,y_train):
  dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
  dataset = dataset.shuffle(100).batch(64,drop_remainder=True)
  return dataset
  
train_dataset = get_dataset(noisy_train[:40000],clean_train[:40000])
test_dataset = get_dataset(noisy_train[40000:],clean_train[40000:])  

In [ ]:
inp = Input(shape=(12000,1))
c1 = Conv1D(2,32,2,'same',activation='relu')(inp)
c2 = Conv1D(4,32,2,'same',activation='relu')(c1)
c3 = Conv1D(8,32,2,'same',activation='relu')(c2)
c4 = Conv1D(16,32,2,'same',activation='relu')(c3)
c5 = Conv1D(32,32,2,'same',activation='relu')(c4)


dc1 = Conv1DTranspose(32,32,1,padding='same')(c5)
conc = Concatenate()([c5,dc1])
dc2 = Conv1DTranspose(16,32,2,padding='same')(conc)
conc = Concatenate()([c4,dc2])
dc3 = Conv1DTranspose(8,32,2,padding='same')(conc)
conc = Concatenate()([c3,dc3])
dc4 = Conv1DTranspose(4,32,2,padding='same')(conc)
conc = Concatenate()([c2,dc4])
dc5 = Conv1DTranspose(2,32,2,padding='same')(conc)
conc = Concatenate()([c1,dc5])
dc6 = Conv1DTranspose(1,32,2,padding='same')(conc)
conc = Concatenate()([inp,dc6])
dc7 = Conv1DTranspose(1,32,1,padding='same',activation='linear')(conc)

model = Model(inp,dc7)


In [15]:
model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 48000, 1)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_43 (Conv1D)  │ (None, 24000, 2)  │         66 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_44 (Conv1D)  │ (None, 12000, 4)  │        260 │ conv1d_43[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_45 (Conv1D)  │ (None, 6000, 8)   │      1,032 │ conv1d_44[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_46 (Conv1D)  │ (None, 3000, 16)  │      4,112 │ conv1d_45[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_47 (Conv1D)  │ (None, 1500, 32)  │     16,416 │ conv1d_46[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_41 │ (None, 1500, 32)  │     32,800 │ conv1d_47[0][0]   │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_36      │ (None, 1500, 64)  │          0 │ conv1d_47[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_42 │ (None, 3000, 16)  │     32,784 │ concatenate_36[0… │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_37      │ (None, 3000, 32)  │          0 │ conv1d_46[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_43 │ (None, 6000, 8)   │      8,200 │ concatenate_37[0… │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_38      │ (None, 6000, 16)  │          0 │ conv1d_45[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_44 │ (None, 12000, 4)  │      2,052 │ concatenate_38[0… │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_39      │ (None, 12000, 8)  │          0 │ conv1d_44[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_45 │ (None, 24000, 2)  │        514 │ concatenate_39[0… │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_40      │ (None, 24000, 4)  │          0 │ conv1d_43[0][0],  │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_transpose_46 │ (None, 48000, 1)  │        129 │ concatenate_40[0… │
│ (Conv1DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_41      │ (None, 48000, 2)  │          0 │ input_layer_7[0]… │
│ (Concatenate)       │                   │            │ conv1d_transpose… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 98,430 (384.49 KB)

 Trainable params: 98,430 (384.49 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.compile(optimizer=Adam(0.002),loss=MeanAbsoluteError())


In [19]:
history = model.fit(train_dataset,epochs=5)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 126s 201ms/step - loss: 0.0115
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 131s 210ms/step - loss: 0.0124
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 134s 215ms/step - loss: 0.0116
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 130s 209ms/step - loss: 0.0116
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 129s 206ms/step - loss: 0.0113


In [25]:
from IPython.display import Audio
Audio(np.squeeze(noisy_train[555].numpy()),rate=16000)

In [23]:
Audio(tf.squeeze(model.predict(tf.expand_dims(tf.expand_dims(noisy_train[555],-1),0))),rate=16000)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
